In [46]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
from selenium import webdriver
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

In [118]:

# id_num = 'random104142'
# seq = 'ccagtggtttagggcctggctaaaaacgct'
NAconc = '0.330'
temp = 30

candidate_file = pd.read_csv('/Users/wenxu/Dropbox (GaTech)/McGrath/LabData/Wen/ProbeDesign/output_11word/blast_results_40.csv')

ids = candidate_file['id']
seqs = candidate_file['sequence']

url = 'https://www.idtdna.com/calc/analyzer'
driver = webdriver.Chrome(executable_path='/Users/wenxu/chromedriver')


def mfold(id_num,seq):
    driver.get(url)
    name_box = driver.find_element_by_name('SEQ_NAME')
    name_box.send_keys(id_num)
    seq_box = driver.find_element_by_name('SEQUENCE')
    seq_box.send_keys(seq)
    NAconc_box = driver.find_element_by_name('NA_CONC')
    NAconc_box.clear()
    NAconc_box.send_keys(NAconc)
    temp_box = driver.find_element_by_name('T')
    temp_box.clear()
    temp_box.send_keys(temp)
    SUBMIT_XPATH = '//input[@type="submit" and @value="Fold DNA"]'
    submit_button = driver.find_element_by_xpath(SUBMIT_XPATH)
    submit_button.click()
    time.sleep(5)
    response = requests.get(driver.current_url)
    html_page = response.content
    parsed_html = BeautifulSoup(html_page,'html5lib')
    all_links = parsed_html.find_all('a')
    hits = []
    for link in all_links:
        if 'Structure' in link.getText():
            hits.append(link.text)
    n_structure = len(hits)-1
    return driver.current_url,n_structure


out_links = []
n_structures = []

for i in range(len(candidate_file)):
    id_num = ids[i]
    seq = seqs[i]
    out_url,n_structure = mfold(id_num,seq)
    out_links.append(out_url)
    n_structures.append(n_structure)
    print(str(i+1) + ': ' + id_num + ' folded. ' + str(n_sturcture) + ' structures found.')
    

In [119]:
df = pd.concat([candidate_file,pd.Series(out_links)],axis=1)
df = pd.concat([df,pd.Series(n_structures)],axis=1)


In [120]:
df.to_csv('mfold_out.csv')